In [127]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 24 18:35:10 2021
@author: Arya
"""
import re
import csv
import nltk
import math
import glob
import numpy
#import pandas
import string
import sklearn
import pandas as pd
from math import log
from sklearn import metrics
from functools import reduce
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.metrics import recall_score
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import normalized_mutual_info_score


In [92]:
#1

corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
""".split("\n")[1:-1]

In [93]:
#2

l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

['simple', 'example', 'with', 'cats', 'and', 'mouse']
['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats']
['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


In [94]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'cheese', 'and', 'another', 'cats', 'dogs', 'mouse', 'simple', 'example', 'with'}


In [95]:
#4
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1
    
for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,cheese,and,another,cats,dogs,mouse,simple,example,with
0,0,1,0,1,0,1,1,1,1
1,0,1,1,1,1,0,1,1,1
2,1,1,1,0,0,1,1,1,1


In [96]:
#4 tf - term frequency

def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [97]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)
print(tf_A)
print(tf_B)
print(tf_C)

{'cheese': 0.0, 'and': 0.16666666666666666, 'another': 0.0, 'cats': 0.16666666666666666, 'dogs': 0.0, 'mouse': 0.16666666666666666, 'simple': 0.16666666666666666, 'example': 0.16666666666666666, 'with': 0.16666666666666666}
{'cheese': 0.0, 'and': 0.14285714285714285, 'another': 0.14285714285714285, 'cats': 0.14285714285714285, 'dogs': 0.14285714285714285, 'mouse': 0.0, 'simple': 0.14285714285714285, 'example': 0.14285714285714285, 'with': 0.14285714285714285}
{'cheese': 0.14285714285714285, 'and': 0.14285714285714285, 'another': 0.14285714285714285, 'cats': 0.0, 'dogs': 0.0, 'mouse': 0.14285714285714285, 'simple': 0.14285714285714285, 'example': 0.14285714285714285, 'with': 0.14285714285714285}


In [98]:
#5 idf - inverse document frequency

def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [99]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
# print(idf)
pd.DataFrame([idf])


,cheese,and,another,cats,dogs,mouse,simple,example,with
0,1.098612,0.0,0.405465,0.405465,1.098612,0.405465,0.0,0.0,0.0


In [100]:
# 6 tf-idf

def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [101]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [102]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,cheese,and,another,cats,dogs,mouse,simple,example,with
0,0.000000,0.0,0.000000,0.067578,0.000000,0.067578,0.0,0.0,0.0
1,0.000000,0.0,0.057924,0.057924,0.156945,0.000000,0.0,0.0,0.0
2,0.156945,0.0,0.057924,0.000000,0.000000,0.057924,0.0,0.0,0.0


In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [104]:
# all_text = """
# Google and Facebook are strangling the free press to death. Democracy is the loser
# Your 60-second guide to security stuff Google touted today at Next '18
# A Guide to Using Android Without Selling Your Soul to Google
# Review: Lenovo’s Google Smart Display is pretty and intelligent
# Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
# Android is better than IOS
# In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
# is a numerical statistic that is intended to reflect
# how important a word is to a document in a collection or corpus.
# It is often used as a weighting factor in searches of information retrieval
# text mining, and user modeling. The tf-idf value increases proportionally
# to the number of times a word appears in the document
# and is offset by the frequency of the word in the corpus
# """.split("\n")[1:-1]

#1
labels = [ "grain", "earn", "acq", "ship", "money-fx", "trade","crude","interest"]
y_true = []
texts = []
openig = open('news.csv', 'r')
reading = csv.reader(openig)

In [105]:
def prep():
    stop_words = set(stopwords.words('english'))

    for line in reading:
    # for field in line:
        get_token = nltk.word_tokenize(line[1])
        stop_words_removed = [w for w in get_token if not w.lower() in stop_words]
        stop_words_removed = []

        for w in get_token:
            if w not in stop_words_removed:
                stop_words_removed.append(w)
        # print(get_token)
        words = [word for word in stop_words_removed if word.isalpha()]

        porter = PorterStemmer()
        stemmed = [porter.stem(word) for word in words]

        y_true.append(labels.index(line[2]))

        string_tokens=' '.join([str(elem) for elem in stemmed])
        texts.append(string_tokens)
    return texts

In [106]:
def textReader(fname):
    doc=open(fname,'rb')
    document = ' '.join(line.decode('utf-8').strip() for line in doc.readlines())
    return document

documents=[]
for files in glob.glob("5501_files/*.txt"):
     documents.append(textReader(files))
          
nltk_tokenizer=RegexpTokenizer(r'\w+')

for index,document in enumerate(documents):
    documents[index]= nltk_tokenizer.tokenize(document)
    
stop_words=set(stopwords.words("english"))
for index,document in enumerate(documents):
    documents[index]=[w for w in document if w not in stop_words ]

ps=PorterStemmer()

all_words=[]

# print (document)

In [107]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

In [108]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing, max_features=3000)
tfidf = tfidf_vectorizer.fit_transform(prep())

In [109]:
kmeans = KMeans(n_clusters=8).fit(tfidf)

In [110]:
lines_for_predicting = texts
y_pred = kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

In [111]:
# def purity_score_1(y_true, y_pred):
#     """Purity score
#         Args:
#             y_true(np.ndarray): n*1 matrix Ground truth labels
#             y_pred(np.ndarray): n*1 matrix Predicted clusters

#         Returns:
#             float: Purity score
#     """
#     # matrix which will hold the majority-voted labels
#     y_voted_labels = np.zeros(y_true.shape)
#     # Ordering labels
#     ## Labels might be missing e.g with set like 0,2 where 1 is missing
#     ## First find the unique labels, then map the labels to an ordered set
#     ## 0,2 should become 0,1
#     labels = np.unique(y_true)
#     ordered_labels = np.arange(labels.shape[0])
#     for k in range(labels.shape[0]):
#         y_true[y_true==labels[k]] = ordered_labels[k]
#     # Update unique labels
#     labels = np.unique(y_true)
#     # We set the number of bins to be n_classes+2 so that 
#     # we count the actual occurence of classes between two consecutive bins
#     # the bigger being excluded [bin_i, bin_i+1[
#     bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

#     for cluster in np.unique(y_pred):
#         hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)
#         # Find the most present label in the cluster
#         winner = np.argmax(hist)
#         y_voted_labels[y_pred==cluster] = winner

#     return accuracy_score(y_true, y_voted_labels)

In [112]:
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # print(contingency_matrix)
    return numpy.sum(numpy.amax(contingency_matrix, axis=0)) / numpy.sum(contingency_matrix) 


In [113]:
purity_score(y_true, y_pred)
c_purity_score = round(purity_score(y_true, y_pred), 5)
print("purity Score is: ", c_purity_score)

purity Score is:  0.77059


In [114]:
adjusted_rand_score(y_true, y_pred)
c_adjusted_rand_score = round(adjusted_rand_score(y_true, y_pred), 5)
print("RI Score is: ", c_adjusted_rand_score)

RI Score is:  0.27014


In [115]:
normalized_mutual_info_score(y_true, y_pred)
c_normalized_mutual_info_score = round(normalized_mutual_info_score(y_true, y_pred), 5)
print("Normalized Mutual Information Score is: ", c_normalized_mutual_info_score)

Normalized Mutual Information Score is:  0.43557


In [116]:
prs = precision_score(y_true, y_pred, average='weighted')
c_prs = round(precision_score(y_true, y_pred, average='weighted'), 5)
print("Precision Score is: ", c_prs)

Precision Score is:  0.79633


In [117]:
rcs = recall_score(y_true, y_pred, average='weighted')
c_rcs = round(recall_score(y_true, y_pred, average='weighted'), 5)
print("Recall Score is: ", c_rcs)

Recall Score is:  0.39211


In [118]:
f1 = (2)*(prs * rcs)/(prs + rcs)
c_f1 = round(f1, 5)
print("F1 Score(1) is: ", c_f1)


F1 Score(1) is:  0.52548


In [126]:
confusion_matrix(y_true, y_pred)

array([[   0,    0,    0,   34,    7,    0,    0,    0],
       [ 229,  681,   57,   50,  234,  455,  442,  692],
       [   2,    0, 1282,  165,  128,   19,    0,    0],
       [   0,    0,   11,   87,   10,    0,    0,    0],
       [   0,    0,    1,  114,  107,    0,    0,    0],
       [   0,    0,    0,  224,   26,    0,    0,    0],
       [   0,    0,   15,  188,   49,    1,    0,    0],
       [   0,    0,    1,   40,  150,    0,    0,    0]], dtype=int64)

In [119]:
# pd.DataFrame([y_true,  y_pred])
# confusion_matrix(y_true, y_pred)

In [120]:
average_link_agglomerative = AgglomerativeClustering(n_clusters=8, linkage='average').fit(tfidf.toarray())

In [121]:
purity_score(y_true, average_link_agglomerative.labels_)
c_purity_score = round(purity_score(y_true, average_link_agglomerative.labels_), 5)
print("Average Link Purity Score is: ", c_purity_score)

adjusted_rand_score(y_true, average_link_agglomerative.labels_)
c_adjusted_rand_score = round(adjusted_rand_score(y_true, average_link_agglomerative.labels_), 5)
print("Average Link RI Score is: ", c_adjusted_rand_score)

normalized_mutual_info_score(y_true,average_link_agglomerative.labels_)
c_normalized_mutual_info_score = round(normalized_mutual_info_score(y_true,average_link_agglomerative.labels_), 5)
print("Average Link Normalized Mutual Information Score is: ", c_normalized_mutual_info_score)

f1_score(y_true, average_link_agglomerative.labels_, average='macro')
print("Average Link F1 Score(1) is: ", round(f1_score(y_true, average_link_agglomerative.labels_, average='macro'),5))

Average Link Purity Score is:  0.69787
Average Link RI Score is:  0.33946
Average Link Normalized Mutual Information Score is:  0.37165
Average Link F1 Score(1) is:  0.12847


In [122]:
complete_link_agglomerative = AgglomerativeClustering(n_clusters=8, linkage='complete').fit(tfidf.toarray())

In [123]:
purity_score(y_true, complete_link_agglomerative.labels_)
c_purity_score = round(purity_score(y_true, complete_link_agglomerative.labels_), 5)
print("Complete Link Purity Score is: ", c_purity_score)

adjusted_rand_score(y_true, complete_link_agglomerative.labels_)
c_adjusted_rand_score = round(adjusted_rand_score(y_true, complete_link_agglomerative.labels_), 5)
print("Complete Link RI Score is: ", c_adjusted_rand_score)

normalized_mutual_info_score(y_true,complete_link_agglomerative.labels_)
c_normalized_mutual_info_score = round(normalized_mutual_info_score(y_true,complete_link_agglomerative.labels_), 5)
print("Complete Link Normalized Mutual Information Score is: ", c_normalized_mutual_info_score)

f1_score(y_true, complete_link_agglomerative.labels_, average='macro')
print("Complete Link F1 Score(1) is: ", round(f1_score(y_true, complete_link_agglomerative.labels_, average='macro'),5))

Complete Link Purity Score is:  0.64025
Complete Link RI Score is:  0.21961
Complete Link Normalized Mutual Information Score is:  0.23629
Complete Link F1 Score(1) is:  0.06759


In [124]:
single_link_agglomerative = AgglomerativeClustering(n_clusters=8, linkage='single').fit(tfidf.toarray())

In [125]:
purity_score(y_true, single_link_agglomerative.labels_)
c_purity_score = round(purity_score(y_true, single_link_agglomerative.labels_), 5)
print("Single Link Purity Score is: ", c_purity_score)

adjusted_rand_score(y_true, single_link_agglomerative.labels_)
c_adjusted_rand_score = round(adjusted_rand_score(y_true, single_link_agglomerative.labels_), 5)
print("Single Link RI Score is: ", c_adjusted_rand_score)

normalized_mutual_info_score(y_true,single_link_agglomerative.labels_)
c_normalized_mutual_info_score = round(normalized_mutual_info_score(y_true,single_link_agglomerative.labels_), 5)
print("Single Link Normalized Mutual Information Score is: ", c_normalized_mutual_info_score)

f1_score(y_true, single_link_agglomerative.labels_, average='macro')
print("Single Link F1 Score(1) is: ", round(f1_score(y_true, single_link_agglomerative.labels_, average='macro'),5))

Single Link Purity Score is:  0.51736
Single Link RI Score is:  0.00195
Single Link Normalized Mutual Information Score is:  0.00517
Single Link F1 Score(1) is:  0.00185
